# Carregamento e exploração dos dados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

base_census = pd.read_csv('./../csv-files/census.csv')


In [ ]:
# Descrição dos dados
base_census.describe()

In [ ]:
# Valores nulos
base_census.isnull().sum()

In [ ]:
# Quantidade de valores de cada classificação
np.unique(base_census['income'], return_counts=True)

In [ ]:
sns.countplot(x = base_census['income'])

In [ ]:
plt.hist(x = base_census['age'])

In [ ]:
plt.hist(x = base_census['education-num'])

In [ ]:
plt.hist(x = base_census['hour-per-week'])

# Tratamento dos dados

Nossa coluna alvo é "income" que está na ultima posição. Porém, o XGBoost necessita que a coluna alvo esteja na primeria posição, entoa iremos reposiciona-la.

In [ ]:
colunas = []
colunas.append('income')
for i in range(len(base_census.columns[:-1])):
    #print(i)
    #print(base_census.columns[i])
    colunas.append(base_census.columns[i])

base_census = base_census[colunas]
base_census    

Tambem é necessario converter as colunas em formato "String" para categorias.

In [ ]:
np.unique(base_census['relationship'])

In [ ]:
print(base_census.info())

As categorias deveriam se tornar numeros, entretando, o algoritimo poderia entender que categoria 5 possui um peso maior que a 1. Por isso utilizaremos o OneHotEncoder - Variaveis dummies

In [ ]:
# 1 0 0 0 0 - Husband
# 0 1 0 0 0 - Not In Family
# 0 0 1 0 0 - Other Relative
# ...

pd.get_dummies(base_census['relationship'])

In [ ]:
# Refatorando colunas
base_census = pd.get_dummies(base_census, prefix = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'inative-country'],
                             columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'inative-country'])
base_census

Coletano colunas previsoras 

In [ ]:
base_census.columns

In [ ]:
X = base_census.iloc[:, 1:len(base_census)].values
X = np.array(X).astype('float32')
X.shape

Formato numpy array

In [ ]:
y = base_census.iloc[:, 0].values
y

Conversao: >=50 = 1 // <=50 = 0

In [ ]:
y_1 = []
for i in y:
    #print(i)
    if i == ' >50K':
        y_1.append(1.0)
    else:
        y_1.append(0.0)
print(y_1)

In [ ]:
# Necessario que estaja no formato flot
Y = np.array(y_1).astype('float32')

Criando selecões de treinamento e de teste

In [ ]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, Y, test_size = 0.3, random_state = 0)


In [ ]:
# X - tResgitros para treinar e testar - Atributos previsores
X_treinamento.shape, X_teste.shape

In [ ]:
# Y - 0 e 1 que indicam as classes
Y_treinamento.shape, Y_teste.shape

# Configurações do SageMaker

In [ ]:
import sagemaker
import boto3
from sagemaker import Session
import sagemaker.amazon.common as smac
import os
import io

session = sagemaker.Session()


In [ ]:
bucket = 'sagemaker-curso-bucket'

subpasta_modelo = 'modelos/census/linear-learner'
subpasta_dataset = 'datasets/census'
key = 'census-train-data'

role = "AmazonSageMaker-ExecutionRole-20240702T173175"

s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Localização da base de dados: ', s3_train_data)
print('Modelo final será salvo em: ', output_location)

Convertendo base de dados:

In [ ]:
buffer = io.BytesIO()
smac.write_numpy_to_dense_tensor(buffer, X_treinamento, Y_treinamento)
buffer.seek(0)

In [ ]:
boto3.resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key)).upload_fileobj(buffer)

# Criação e treinamento do linear Learner

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
container = sagemaker.image_uris.retrieve(framework = 'linear-learner', region=boto3.Session().region_name)

In [ ]:
linear = sagemaker.estimator.Estimator(image_uri = container,
                                       role = role,
                                       instance_count = 1,
                                       instance_type = 'ml.m4.xlarge',
                                       output_path = output_location,
                                       sagemaker_session = session,
                                       use_stop_instances = True)

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html
hyper_parameters = {
    'feature_dim' : 108,
    'predictor_type' : 'binary_classifier',
    'num_models' : 8
}

linear.set_hyperparameters(hyper_parameters)

# Binary Classifier para casos de classificação

In [ ]:
linear.fit({'train': s3_train_data})

# Deploy, previsões e avaliação

In [ ]:
linear_classifier = linear.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
linear_classifier.serializer = CSVSerializer()
linear_classifier.deserializer = JSONDeserializer()

In [ ]:
X_teste.shape

In [ ]:
results = linear_classifier.predict(X_teste)
results

In [ ]:
previsoes = np.array([r['predicted_label'] for r in results['predictions']])
previsoes

In [ ]:
y_teste = np.array(Y_teste).astype(int)
y_teste

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
accuracy_score(y_teste, previsoes)

In [ ]:
cm = confusion_matrix(y_teste, previsoes)
cm

In [ ]:
print(classification_report(y_teste, previsoes))